# 🔍 Anomaly Detection

**Author**: Data Science Master System  
**Difficulty**: ⭐⭐⭐ Advanced  
**Time**: 45 minutes  
**Prerequisites**: 21_time_series_forecasting

## Learning Objectives
- Statistical methods (Z-score, IQR)
- Isolation Forest
- Autoencoders for anomalies
- Streaming anomaly detection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

np.random.seed(42)

## 1. Generate Data with Anomalies

In [ ]:
# Normal data
n_samples = 300
X_normal = np.random.randn(n_samples, 2)

# Inject anomalies
n_anomalies = 20
X_anomalies = np.random.uniform(low=-4, high=4, size=(n_anomalies, 2))

X = np.vstack([X_normal, X_anomalies])
y_true = np.array([0]*n_samples + [1]*n_anomalies)  # 0=normal, 1=anomaly

plt.figure(figsize=(8, 6))
plt.scatter(X[:n_samples, 0], X[:n_samples, 1], label='Normal', alpha=0.6)
plt.scatter(X[n_samples:, 0], X[n_samples:, 1], label='Anomaly', color='red', s=100)
plt.legend()
plt.title('Data with Anomalies')
plt.show()

## 2. Isolation Forest

In [ ]:
iso_forest = IsolationForest(contamination=0.1, random_state=42)
y_pred = iso_forest.fit_predict(X)

# Convert to binary (1=normal, -1=anomaly → 0=normal, 1=anomaly)
y_pred_binary = (y_pred == -1).astype(int)

# Accuracy
accuracy = (y_pred_binary == y_true).mean()
print(f"Isolation Forest Accuracy: {accuracy:.2%}")

# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(X[y_pred == 1, 0], X[y_pred == 1, 1], label='Normal (pred)', alpha=0.6)
plt.scatter(X[y_pred == -1, 0], X[y_pred == -1, 1], label='Anomaly (pred)', color='red', s=100)
plt.legend()
plt.title('Isolation Forest Predictions')
plt.show()

## 3. Statistical Methods

In [ ]:
def z_score_anomalies(data, threshold=3):
    """Detect anomalies using Z-score."""
    scaler = StandardScaler()
    z_scores = np.abs(scaler.fit_transform(data))
    return (z_scores > threshold).any(axis=1)

def iqr_anomalies(data, k=1.5):
    """Detect anomalies using IQR."""
    anomalies = np.zeros(len(data), dtype=bool)
    for col in range(data.shape[1]):
        Q1, Q3 = np.percentile(data[:, col], [25, 75])
        IQR = Q3 - Q1
        anomalies |= (data[:, col] < Q1 - k*IQR) | (data[:, col] > Q3 + k*IQR)
    return anomalies

print(f"Z-score anomalies: {z_score_anomalies(X).sum()}")
print(f"IQR anomalies: {iqr_anomalies(X).sum()}")

## 4. Method Comparison

In [ ]:
comparison = pd.DataFrame({
    'Method': ['Z-Score', 'IQR', 'Isolation Forest', 'LOF', 'Autoencoder'],
    'Type': ['Statistical', 'Statistical', 'ML', 'ML', 'Deep Learning'],
    'Best For': ['Univariate', 'Robust/outliers', 'High-dim', 'Local anomalies', 'Complex patterns'],
    'Streaming': ['Yes', 'Yes', 'No', 'No', 'Partial']
})

display(comparison)

## 🎯 Key Takeaways
1. Isolation Forest: Best general-purpose
2. Z-score: Fast, simple, univariate
3. Set contamination based on domain knowledge
4. Combine methods for robust detection

**Next**: 23_recommender_systems.ipynb